In [ ]:
import numpy as np
import tensorflow as tf
import cv2, os
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping


In [ ]:
from google.colab import drive  # on colab

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json

path = '/content/drive/MyDrive/Colab Notebooks/Project'

with open(f'{path}/photo_data.json', 'r', encoding='utf-8') as f:
    post_data = json.load(f)

In [ ]:
images, exposures = [], []
for img_name in os.listdir(f'{path}/photos'):
    img = cv2.imread(f'{path}/photos/{img_name}')
    img = cv2.resize(img, (224, 224,))
    images.append(img)
    
    img_id = img_name.split('_')[0]
    photo_views = post_data[img_id]['photo_views']
    impressions_unique = post_data[img_id]['impressions_unique']
    r = photo_views / impressions_unique
    exposures.append(r)


In [ ]:
sorted_data = np.sort(exposures)

Q1 = np.percentile(sorted_data, 25)
Q2 = np.percentile(sorted_data, 50)
Q3 = np.percentile(sorted_data, 75)

split1 = sorted_data[0]
split2 = Q1
split3 = Q2
split4 = Q3
split5 = sorted_data[-1]

exposure_labels = np.zeros_like(exposures)
exposure_labels[(exposures >= split1) & (exposures < split2)] = 0
exposure_labels[(exposures >= split2) & (exposures < split3)] = 1
exposure_labels[(exposures >= split3) & (exposures < split4)] = 2
exposure_labels[(exposures >= split4) & (exposures <= split5)] = 3

print(split2, split3, split4)

0.0039728349886637305 0.015248001220329647 0.045274756321514335


In [ ]:
X = np.array(images)/255.
y = np.array(exposure_labels)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
y_train = to_categorical(y_train, num_classes=4)
y_test = to_categorical(y_test, num_classes=4)

In [ ]:
from tensorflow.keras.applications import InceptionV3

base_model = InceptionV3(input_shape=(224, 224, 3), include_top=False, weights='imagenet') 

for layer in base_model.layers:
    layer.trainable = False  #true or mobilenet

model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(512, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.2))
model.add(Dense(4, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 5, 5, 2048)        21802784  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 512)               1049088   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                        

In [ ]:
from keras.optimizers import RMSprop

optimizer = Adam(learning_rate=0.001) 
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test), validation_freq=1)# , callbacks=[early_stopping])

In [ ]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)

# Print the test set accuracy
print('Test set accuracy:', accuracy)

In [ ]:
# model.save('my_model.h5')